In [0]:
# 05 - KPI Inicial

# COMMAND ----------
from pyspark.sql.functions import col, count, when, isnan, avg
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve
import joblib
import numpy as np

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

# COMMAND ----------
# 📌 Cargar tablas
df_raw = spark.table("datalottery.lotterybets.lottery_bets_dirty")

print(f"📊 Registros originales: {df_raw.count()}")

# COMMAND ----------
# 📊 Calcular nulos
from pyspark.sql.types import NumericType

def contar_nulos(df):
    numeric_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, NumericType)]
    non_numeric_cols = [f.name for f in df.schema.fields if not isinstance(f.dataType, NumericType)]

    nulos_num = df.select([
        count(when(col(c).isNull() | isnan(col(c)), c)).alias(c) for c in numeric_cols
    ])
    nulos_no_num = df.select([
        count(when(col(c).isNull(), c)).alias(c) for c in non_numeric_cols
    ])
    return nulos_num.join(nulos_no_num).toPandas().T

nulos_raw = contar_nulos(df_raw).rename(columns={0: 'nulos_raw'})

# COMMAND ----------
# 📌 Preparar features para más análisis
df_pd = df_raw.select(
    "bets_last_7d", "win_rate_last_30d", "ip_risk", "geo_risk",
    "num_picks", "stake_amount", "suspicious"
).toPandas()


# COMMAND ----------
# 📊 Distribución de la variable objetivo
sns.countplot(data=df_pd, x="suspicious")
plt.title("Distribución de registros sospechosos")
plt.show()


# 📊 Distribución de bets_last_7d
sns.histplot(df_pd["bets_last_7d"], bins=20, kde=True)
plt.title("Distribución de apuestas últimos 7 días")
plt.show()

# 📊 Distribución de win_rate_last_30d
sns.histplot(df_pd["win_rate_last_30d"], bins=20, kde=True)
plt.title("Distribución de tasa de victoria últimos 30 días")
plt.show()


# COMMAND ----------
# 📌 KPIs resumen
kpis = {
    "Registros originales": df_raw.count(),
    "Tasa sospechosos (%)": round(100*df_pd["suspicious"].mean(),2),
    "Promedio stake_amount": round(df_pd["stake_amount"].mean(),2),
    "Promedio bets_last_7d": round(df_pd["bets_last_7d"].mean(),2),
    "Promedio win_rate_30d": round(df_pd["win_rate_last_30d"].mean(),3),
}

# Crear DataFrame desde el diccionario
kpis_df = pd.DataFrame(list(kpis.items()), columns=["KPI", "Valor"])

# Asegurar que la columna 'Valor' esté en formato string (por seguridad al mostrar)
kpis_df["Valor"] = kpis_df["Valor"].astype(str)

# Mostrar resultados en notebook (Databricks)
display(kpis_df)
